# Q2

In [16]:
import numpy as np 
import matplotlib.pyplot as plt

B1 = 0.6965325 
B2 = 0.4083099 
B3 = 0.8968766 
C1 = 4.368309e-03 
C2 = 1.394999e-02 
C3 = 9.793399e+01
Bs = np.array([B1, B2, B3])
Cs = np.array([C1, C2, C3])

B1g = 0.7083952 
B2g = 0.4203993 
B3g = 0.8663412 
C1g = 7.290464e-03 
C2g = 1.050294e-02 
C3g = 9.793428e+01
Bgs = np.array([B1g, B2g, B3g])
Cgs = np.array([C1g, C2g, C3g])

def get_n(wl, x):
    wl = np.atleast_1d(wl)

    B = (Bs + x * (Bgs - Bs))
    C = (Cs + x * (Cgs - Cs))
    B = B[:, np.newaxis]
    C = C[:, np.newaxis]
    wl2 = wl**2
    wl2 = wl2[np.newaxis, :]
    num = B * wl2
    denum = wl2 - C
    sellmeier_sum = (num / denum).sum(axis=0)
    
    return np.sqrt(1 + sellmeier_sum)

wls = np.linspace(0.4, 2.4, 100)
fig, ax = plt.subplots(constrained_layout=True, figsize=(9, 5))
ax.plot(wls, get_n(wls, x=0), label='x=0')
ax.plot(wls, get_n(wls, x=0.5), label='x=0.5')
ax.plot(wls, get_n(wls, x=1), label='x=1')
ax.set_xlabel('Wavelength (um)', fontsize=14)
ax.set_ylabel('refractive index', fontsize=14)
ax.legend()
fig.show()

# Q3 

In [21]:
wl = 1.55  # um
k0 = 2*np.pi / wl  # 1 / um 
a = 4.4  # um 

n0 = get_n(wl, x=0)[0]
n1 = get_n([wl], x=0.5)[0]
V = k0*a*np.sqrt(n1**2 - n0**2)

print(f'{n0=}')
print(f'{n1=}')
print(f'{V=}')

n0=1.4442785377765517
n1=1.4486114568577975
V=1.996903580324177


In [38]:
from scipy.special import j0 as J0, j1 as J1, k0 as K0, k1 as K1
from scipy.optimize import fsolve

# this needs to be zero 
def kappa_eq(kappa):
    # Avoid division by zero if kappa is near zero
    # (In practice, for LP01, we wouldn't expect kappa=0 to be the solution anyway)
    if abs(kappa) < 1e-15:
        return 1e15  # large number, effectively not the root we want
    
    gamma = np.sqrt((k0**2 * (n1**2 - n0**2)) - kappa**2)
    LHS = J0(kappa*a) / (kappa*a * J1(kappa*a))
    RHS = K0(gamma*a) / (gamma*a * K1(gamma*a))
    
    return LHS - RHS 

def plot_LHS_RHS(kappa_min, kappa_max, num_points=1000):
    kappa_values = np.linspace(kappa_min, kappa_max, num_points)
    LHS = []
    RHS = []
    
    for kappa in kappa_values:
        try:
            gamma_sq = (k0**2 * (n1**2 - n0**2)) - kappa**2
            if gamma_sq < 0:
                LHS.append(np.nan)
                RHS.append(np.nan)
                continue
            gamma = np.sqrt(gamma_sq)
            lhs_val = J0(kappa * a) / (kappa * a * J1(kappa * a))
            rhs_val = K0(gamma * a) / (gamma * a * K1(gamma * a))
            LHS.append(lhs_val)
            RHS.append(rhs_val)
        except:
            LHS.append(np.nan)
            RHS.append(np.nan)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(kappa_values, LHS, label='LHS: $J_0(\\kappa a)/(\\kappa a J_1(\\kappa a))$')
    ax.plot(kappa_values, RHS, label='RHS: $K_0(\\gamma a)/(\\gamma a K_1(\\gamma a))$')
    ax.set_xlabel(r'$\kappa$ (1/μm)')
    ax.set_ylabel('Function Value')
    ax.set_title('Characteristic Equation for LP$_{01}$ Mode')
    ax.legend()
    ax.grid(True)
    ax.set_ylim(-0.5, 4)  # Adjust as necessary
    fig.show()
 

kappa_min = 1e-3
kappa_max = k0*np.sqrt(n1**2 - n0**2) + 1e-2
plot_LHS_RHS(kappa_min, kappa_max, num_points=100)

kappa = fsolve(kappa_eq, x0=0.2)[0]
print(f'{kappa=}')
beta = np.sqrt(k0**2*n1**2 - kappa**2)
print(f'{beta=} 1/um')
gamma = np.sqrt(k0**2 * (n1**2 - n0**2) - kappa**2)
print(f'{gamma=} 1/um')
print(f'{k0*n0=} 1/um')
print(f'{k0*n1=} 1/um')

kappa=0.347075450365457
beta=5.861923909260839 1/um
gamma=0.2924225386466455 1/um
k0*n0=5.854625605182224 1/um
k0*n1=5.872189820348987 1/um


In [42]:
fig, ax = plt.subplots()
r = np.linspace(0, 12, 1000)
normalization = J0(kappa * a) / K0(gamma * a)
E = np.where(r <= a, J0(kappa * r), normalization*K0(gamma * r))
intensity = np.abs(E)**2 

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(r, intensity, label='Intensity |E(r)|²')
ax.axvline(a, color='red', linestyle='--', label='Core Radius (a)')
ax.set_xlabel('r (μm)')
ax.set_ylabel('Intensity')
ax.set_title('Intensity Profile of LP$_{01}$ Mode')
ax.legend()
ax.grid(True)
fig.show()

# Q4

In [54]:
from scipy.optimize import curve_fit

gaussian = lambda r, w: np.exp(-r**2/w**2)
popt, pcov = curve_fit(gaussian, r, E)
w = popt[0]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(9, 4))
axes[0].plot(r, np.abs(E)**2, label='original')
axes[0].plot(r, np.abs(gaussian(r, w))**2, '--', label='fit')


axes[1].plot(r, np.abs(E)**2, label='original')
axes[1].plot(r, np.abs(gaussian(r, w))**2, '--', label='fit')
axes[1].set_yscale('log')
for ax in axes:
    ax.axvline(a, color='red', linestyle='--', label='Core Radius (a)')
    ax.legend()
    ax.set_xlabel('r (um)')
    ax.set_ylabel('Intensity (a.u.)')
fig.show()

In [57]:
print(f'{a=}')
print(f'{w=}')
print(f'{2*w=}')
print(f'{w/a=}')

a=4.4
w=5.409109003608509
2*w=10.818218007217018
w/a=1.2293429553655701
